<a href="https://colab.research.google.com/github/buggytanmoy77/Chaos/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation of necessary libraries and tools

In [2]:
!pip install -q diffusers transformers accelerate torch xformers
!pip install -q clip-interrogator safetensors huggingface_hub
!pip install -q git+https://github.com/huggingface/transformers.git

## Setting up CLIP

In [ ]:
!pip install clip-interrogator==0.6.0

from PIL import Image
from clip_interrogator import Config, Interrogator

ci = Interrogator(Config(
    clip_model_name="ViT-L-14/openai",
    device="cuda"
))

def get_prompt(image_path):
    image = Image.open(image_path).convert("RGB")
    return ci.interrogate(image)

## Download and Initialization of SD model

In [ ]:
!wget https://civitai.com/api/download/models/128713 -O custom_model.safetensors

from diffusers import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_single_file(
    "custom_model.safetensors",
    torch_dtype=torch.float16,
    safety_checker=None
).to("cuda")